# **Cell 1. Environment setup and experiment configuration**

This cell initializes the computational environment and defines the experimental setting used throughout the analysis. It imports all required project modules and dependencies, specifies the data source, and sets the core experimental condition via the time period (PERIOD) and road type filter (TYPE_FILTER).

To reproduce different results reported in the study, only the values of PERIOD and TYPE_FILTER need to be modified. All subsequent steps adapt automatically to these settings.

In [ ]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import numpy as np
import pandas as pd

from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

from modules.data_utils import load_and_preprocess
from modules.model_utils import train_model, cross_val_r2
from modules.shap_stability_utils import shap_stability_check
from modules.shap_utils import compute_shap
from modules.draw_figure import plot_summary, plot_dependence

from modules.config import DTYPE, FEATURE_COLS, DEPENDENCE_LIMITS, select_xgb_params
from modules.io_utils import get_out_dirs, safe_feature_filename


# ---- Change these two to switch experiments ----
DATA = Path("data.csv")
PERIOD = "night"          # "day" or "night"
TYPE_FILTER = None  # None or "motorway"/"trunk"/"primary"/"secondary"/"tertiary"/"residential"


# ---- Outputs: auto folder naming by (period, type_filter) ----
RUN_DIR, FIG_DIR, TAB_DIR, MODEL_DIR = get_out_dirs(
    base_dir=Path("outputs"),
    period=PERIOD,
    type_filter=TYPE_FILTER,
    make=True,
)

# ---- Model hyperparameters: auto selected ----
params = select_xgb_params(PERIOD, TYPE_FILTER)

# **Cell 2. Data loading and preprocessing**

In [ ]:
X, X_display, y = load_and_preprocess(
    DATA,
    dtype=DTYPE,
    feature_cols=FEATURE_COLS,
    group_col="car_id",
    fe_residualize=True,
    period=PERIOD,
    cal="all",
    type_filter=TYPE_FILTER,
    type_col="type",
)
print(f"Records after filtering: {len(X):,}")

# **Cell 3：Cross-validation performance assessment**

In [ ]:
# ---- Cross-validation to assess model robustness ----
cv_scores, cv_mean, cv_std = cross_val_r2(
    X,
    y,
    cv=5,
    random_state=42,
    early_stopping_rounds=100,   # keep consistent with final training
    **params
)

print("Cross-validation R² scores (each fold):", cv_scores)
print(f"Cross-validation mean R²: {cv_mean:.4f} ± {cv_std:.4f}")

# **Cell 4. Final model training and evaluation**

The final model is trained using a single train–validation–test split, with early stopping applied on the validation set. This trained model is used for all subsequent SHAP analyses and visualizations.

In rare cases, model performance may appear suboptimal due to an unfavourable random split, which can lead to premature early stopping when the validation subset is not fully representative. This behaviour reflects sampling variability rather than a systematic model issue.
If such a case occurs, adjusting the random_state parameter and re-running the training step is sufficient to recover stable performance.

The final trained XGBoost model is saved to disk to ensure reproducibility of all downstream SHAP analyses without retraining.

In [ ]:
# ---- Final model training with train/validation/test split ----
model, X_train, X_valid, X_test, y_train, y_valid, y_test = train_model(
    X,
    y,
    test_size=0.1,
    valid_size=0.1,
    random_state=42,
    early_stopping_rounds=100,
    **params
)

# ---- Performance on validation and test sets ----
r2_valid = r2_score(y_valid, model.predict(X_valid))
r2_test = r2_score(y_test, model.predict(X_test))

print(f"Validation R²: {r2_valid:.4f}")
print(f"Test R²: {r2_test:.4f}")

# ---- Additional error metrics on the test set ----
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)

print(f"Test RMSE: {rmse:.4f}")
print(f"Test MAE: {mae:.4f}")

# ---- Save final trained XGBoost model (for reproducibility) ----
model_path = MODEL_DIR / "xgb_model.json"
model.save_model(model_path)

print("Final XGBoost model saved to:", model_path)

# **Cell 5. SHAP stability assessment**

This cell evaluates the robustness of feature-attribution results by repeating SHAP computation on multiple resampled subsets and comparing the resulting feature-importance rankings. Stability is quantified using (i) Spearman rank correlation of mean absolute SHAP importances across repetitions, and (ii) the overlap rate among the top-K most important features.

In [ ]:
# ---- SHAP stability assessment via repeated sampling ----
res = shap_stability_check(
    model,
    X_train,
    X_test,
    explain_n=20000,        # number of samples explained in each repetition
    repeats=5,              # number of repetitions for stability estimation
    strata_cols=[],         # stratification columns (empty = random sampling)
    use_fast_shap=True,     # use fast SHAP approximation for efficiency
    background_n=800,       # only used when use_fast_shap=False
    topk=10,                # top-K features used for overlap calculation
    random_seed=42          # random seed for reproducibility
)

print(
    "SHAP stability (Spearman ρ across repetitions): "
    f"{res['rho_list']} | mean ± sd = {res['rho_mean']:.3f} ± {res['rho_std']:.3f}"
)

print(
    f"Top-{res['topk']} feature overlap ratios: "
    f"{res['topk_overlap_list']} | mean = {res['topk_overlap_mean']:.3f}"
)

# **Cell 6：Final SHAP attribution and feature importance**

In [ ]:
# ---- Final SHAP attribution on a subsample of the test set ----
X_test_shap = X_test.sample(
    n=min(len(X_test), 5000),
    random_state=42
)

explainer, shap_values = compute_shap(
    model,
    X_train,
    X_test_shap,
    background_size=1000
)

# ---- Aggregate SHAP values across samples ----
mean_shap = shap_values.mean(axis=0)
mean_abs_shap = np.abs(shap_values).mean(axis=0)

shap_importance_df = pd.DataFrame({
    "feature": X_test_shap.columns,
    "mean_shap": mean_shap,
    "mean_abs_shap": mean_abs_shap,
    "percentage": mean_abs_shap / mean_abs_shap.sum() * 100,
}).sort_values(by="mean_abs_shap", ascending=False)

print("Top features by mean absolute SHAP value:")
print(shap_importance_df.head(20))

out_csv = TAB_DIR / "shap_importance.csv"
shap_importance_df.to_csv(out_csv, index=False, encoding="utf-8-sig")

print("SHAP importance table saved to:", out_csv)

# ****# Cell 7：Visualization of SHAP results****

In [ ]:
X_display_test = X_display.loc[X_test_shap.index]

# Summary plot
summary_path = FIG_DIR / "shap_summary.png"
plot_summary(shap_values, X_test_shap, str(summary_path))
print("Saved:", summary_path)

# Dependence plots (loop)
for feat, lim in DEPENDENCE_LIMITS.items():
    fname = f"{safe_feature_filename(feat)}_dependence.png"
    out_path = FIG_DIR / fname
    plot_dependence(
        shap_values,
        X_test_shap,
        feat,
        filename=str(out_path),
        display_features=X_display_test,
        xlim=lim.get("xlim"),
        ylim=lim.get("ylim"),
    )

print("Saved dependence plots to:", FIG_DIR)
